# PolyFit - C Code

C-Code embedded from: https://github.com/natedomin/polyfit.git


2020-06-16 ug

2020-11-20 ug

## Clone repository

to later delete the '.\polyfit_C_code'

Git clone without .git directory https://stackoverflow.com/questions/11497457/git-clone-without-git-directory

In [ ]:
!git clone --depth=1 --branch=master https://github.com/natedomin/polyfit.git polyfit_C_code

“rm -rf” equivalent for Windows?
https://stackoverflow.com/questions/97875/rm-rf-equivalent-for-windows

In [ ]:
!rmdir /s /q .\polyfit_C_code\.git

## Prepare

In [ ]:
import numpy as np

## C Code implementation

polyfit.c /.h

https://github.com/natedomin/polyfit

int polyfit(const double* const dependentValues,
            const double* const independentValues,
            unsigned int        countOfElements,
            unsigned int        order,
            double*             coefficients);


In [ ]:
%load_ext cython

In [ ]:
%%writefile polyfit_C.pyx

cdef extern from "polyfit_C_code/polyfit.h":
    int polyfit(const double* const dependentValues,
                const double* const independentValues,
                unsigned int        countOfElements,
                unsigned int        order,
                double*             coefficients);
import numpy as np

def polyfit_C(x, y, coeff, order): # 'arr' is a one-dimensional numpy array

    if not x.flags['C_CONTIGUOUS']:
        x = np.ascontiguousarray(x) # Makes a contiguous copy of the numpy array.
    if not y.flags['C_CONTIGUOUS']:
        y = np.ascontiguousarray(y) # Makes a contiguous copy of the numpy array.
    if not coeff.flags['C_CONTIGUOUS']:
        coeff = np.ascontiguousarray(coeff) # Makes a contiguous copy of the numpy array.
        
    cdef double[::1] x_memview = x
    cdef double[::1] y_memview = y
    cdef double[::1] coeff_memview = coeff
    cdef int order_tmp = order
    
       
    polyfit(&x_memview[0],&y_memview[0], y_memview.shape[0], order_tmp, &coeff_memview[0]);
               
    return coeff

In [ ]:
%%writefile setup_polyfit_c.py
from distutils.core import setup, Extension
from Cython.Build import cythonize
ext = Extension('Cython_Polyfit_C', language="c", sources = ['polyfit_C.pyx', 'polyfit_C_code/polyfit.c'])
setup(name="Cython_Polyfit_C", ext_modules = cythonize([ext]))

In [ ]:
!python setup_polyfit_c.py build_ext --inplace

In [ ]:
import Cython_Polyfit_C

These inputs should result in the following approximate coefficients:

         0.5           2.5           1.0        3.0 
         
    y = (0.5 * x^3) + (2.5 * x^2) + (1.0 * x) + 3.0 

In [ ]:
x_test1 = np.array([12.0, 77.8, 44.1, 23.6, 108.2])
y_test1 = np.array([1239.00, 250668.38, 47792.19, 7991.13, 662740.98])
order = 3
coeff = np.zeros(4)

#polyfit_C(x, y, coeff, order)

coeff = Cython_Polyfit_C.polyfit_C(x_test1, y_test1, coeff, order)
coeff

# Clean up

In [ ]:
import os
import shutil

def deleteFile(FileName):
    print(f'deleting file {FileName}', end ='')
    try:
        os.remove(FileName)
        print(' done')
    except OSError:
        print(' skipped')
def deleteFolder(FolderPathName):
    # https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder
    print('deleting folder %s'%FolderPathName,end='')
    try:
        shutil.rmtree(FolderPathName)   # , ignore_errors=True
        print(' done')
    except OSError:
        print(' skipped')

In [ ]:
fileList = ['setup_polyfit_c.py',
            'polyfit_C.c',
            'polyfit_C.pyx']
for file in fileList:
    deleteFile(file)
deleteFolder(r'.\build')
deleteFolder(r'.\polyfit_C_code')